In [2]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
!unzip project2026.zip
!ls -l

--2025-11-10 21:12:25--  https://www.lamsade.dauphine.fr/~cazenave/project2026.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 138578548 (132M) [application/zip]
Saving to: ‘project2026.zip’

project2026.zip     100%[===================>] 132.16M  21.2MB/s    in 7.0s    

2025-11-10 21:12:33 (18.9 MB/s) - ‘project2026.zip’ saved [138578548/138578548]

Archive:  project2026.zip
  inflating: games.data              
  inflating: golois.cpython-312-x86_64-linux-gnu.so  
total 665408
-rw-r--r-- 1 root root 542497580 Oct  7  2022 games.data
-rwxr-xr-x 1 root root    284672 Oct  1 15:09 golois.cpython-312-x86_64-linux-gnu.so
-rw-r--r-- 1 root root 138578548 Oct  1 20:02 project2026.zip
drwxr-xr-x 1 root root      4096 Nov  7 14:30 sample_data


# Import

In [3]:
# ================================================================
# 📦 IMPORTS GÉNÉRAUX
# ================================================================
import os, gc
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from google.colab import files  # pour le téléchargement auto
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
import golois  # Librairie C++ fournie (jeu de données Go)
from tensorflow.keras import layers, regularizers
from tensorflow import keras
import gc

import json
import matplotlib.pyplot as plt
from google.colab import files



# Utilitaires internes (si tu veux factoriser ton code)
# from utils.model_blocks import dw_conv_block, se_block, attention_residual_block
# from utils.training_utils import moving_average, save_plots


In [18]:
# ================================================================
# ✅ GO-SEPARABLE RESNET — ~100k params (≈99 932) — golois compatible
# ================================================================
import os, gc, shutil, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from datetime import datetime
import golois

# -----------------------------
# GPU safety (Colab)
# -----------------------------
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("✅ GPU activé (memory growth).")
    except Exception as e:
        print("⚠️ GPU memory growth non appliqué:", e)
else:
    print("⚠️ Aucun GPU détecté — CPU utilisé.")

# ================================================================
# 🔧 PARAMÈTRES
# ================================================================
trainer = "AdamH"
base_title = "GoSepResNet_100k"
date_str = datetime.now().strftime("%Y%m%d")

planes = 31
moves  = 361
N      = 10000
epochs = 60
batch  = 128

# 🎯 Calibrage précis pour ~100k
filters            = 110          # nb de canaux dans le backbone
num_blocks         = 3            # nb de blocs résiduels
se_reduction       = 10           # squeeze ratio
value_dense_units  = 64

l2_reg        = 1e-4
learning_rate = 1e-4
label_smooth  = 0.10

# ------------------------------------------------
# 📁 Dossier de sortie
# ------------------------------------------------
model_name  = f"{date_str}_{trainer}_{base_title}_ep{epochs}_bs{batch}_lr{learning_rate}"
output_root = "output"
output_dir  = os.path.join(output_root, model_name)
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)
print(f"🧾 Modèle : {model_name}")
print(f"📂 Dossier : {output_dir}")

# ================================================================
# 📊 BUFFERS (mock init) — golois les remplit ensuite
# ================================================================
input_data = np.random.randint(2, size=(N, 19, 19, planes)).astype('float32')
policy     = keras.utils.to_categorical(np.random.randint(moves, size=N), num_classes=moves).astype('float32')
value      = np.random.randint(2, size=(N,)).astype('float32')
end        = np.random.randint(2, size=(N, 19, 19, 2)).astype('float32')
groups     = np.zeros((N, 19, 19, 1), dtype='float32')

print("🔍 Test golois.getValidation()…")
golois.getValidation(input_data, policy, value, end)
print("✅ golois prêt.\n")

# ================================================================
# 🧱 BLOCS
# ================================================================
BN = lambda: layers.BatchNormalization(momentum=0.95, epsilon=1e-5)

def sep_conv_block(x, filters, l2_reg, stride=1):
    """SeparableConv2D (depthwise + pointwise) + BN + ReLU (sans bias)."""
    x = layers.SeparableConv2D(
        filters,
        kernel_size=3,
        strides=stride,
        padding='same',
        activation=None,
        depthwise_regularizer=regularizers.l2(l2_reg),
        pointwise_regularizer=regularizers.l2(l2_reg),
        use_bias=False
    )(x)
    x = BN()(x)   # BN ajoute 4F params (2 trainables + 2 non-trainables)
    x = layers.ReLU()(x)
    return x

def se_block(x, reduction=10):
    """Squeeze-Excite (léger)"""
    c = int(x.shape[-1])
    s = layers.GlobalAveragePooling2D()(x)
    s = layers.Dense(max(c // reduction, 1), activation='relu')(s)
    s = layers.Dense(c, activation='sigmoid')(s)
    s = layers.Reshape((1, 1, c))(s)
    return layers.Multiply()([x, s])

def residual_separable_block(x, filters, l2_reg, reduction):
    """Bloc résiduel : 2x SeparableConv2D + SE + skip."""
    shortcut = x
    x = sep_conv_block(x, filters, l2_reg)
    x = sep_conv_block(x, filters, l2_reg)
    x = se_block(x, reduction=reduction)
    x = layers.Add()([shortcut, x])
    x = layers.ReLU()(x)
    return x

# ================================================================
# 🧠 MODÈLE
# ================================================================
inp = keras.Input(shape=(19, 19, planes), name='board')

# Stem 1x1 (sans bias) + BN + ReLU
x = layers.Conv2D(
    filters, 1, padding='same', activation=None,
    kernel_regularizer=regularizers.l2(l2_reg),
    use_bias=False
)(inp)
x = BN()(x)
x = layers.ReLU()(x)

# 3 blocs résiduels séparables + SE
for i in range(num_blocks):
    x = residual_separable_block(x, filters, l2_reg, se_reduction)
    if i % 2 == 1:
        x = layers.Dropout(0.10)(x)

# --- Policy head (⚠️ SANS Dense(361) !) ---
# 1x1 conv (sans bias) → 361 logits après flatten → softmax
p = layers.Conv2D(1, 1, padding='same', activation=None,
                  kernel_regularizer=regularizers.l2(l2_reg),
                  use_bias=False)(x)         # params = filters (110)
p = layers.Flatten()(p)                      # 19*19 = 361
policy_head = layers.Activation('softmax', name='policy')(p)

# --- Value head (compact) ---
v = layers.GlobalAveragePooling2D()(x)       # vecteur de taille 110
v = layers.Dense(value_dense_units, activation='relu',
                 kernel_regularizer=regularizers.l2(l2_reg))(v)  # 110*64 + 64 = 7104
v = layers.Dropout(0.15)(v)
value_head = layers.Dense(1, activation='tanh', name='value')(v)  # 64*1 + 1 = 65

model = keras.Model(inputs=inp, outputs=[policy_head, value_head])
model.summary()

# ------------------------------------------------
# 🔢 Vérification stricte du budget de params
# Estimation théorique (incluant BN trainables + non-trainables):
# - Stem: Conv1x1 (31*110=3410) + BN (4*110=440)  -> 3850
# - Par bloc (x3):
#     2×(SepConv: 110*110 + 9*110 = 12100 + 990; BN: 4*110 = 440) = 2×(130? + 440) = 2×(130? wait) = 2×(13090+440)=2×13530=27060
#     SE: (110→11→110) = 1221 + 1320 = 2541
#     Total bloc ≈ 27060 + 2541 = 29601
#   3 blocs ≈ 88803
# - Policy 1x1 (sans bias): 110
# - Value: 110→64 (7104) + 64→1 (65) = 7169
# TOTAL ≈ 3850 + 88803 + 110 + 7169 = 99 932
# ------------------------------------------------
total_params = model.count_params()
print(f"🔢 Total params (Keras): {total_params}")
assert 95_000 <= total_params <= 105_000, f"❌ Hors cible: {total_params} (attendu ≈100k)"

# ================================================================
# ⚙️ COMPILE + CALLBACKS
# ================================================================
optimizer    = keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
policy_loss  = keras.losses.CategoricalCrossentropy(label_smoothing=label_smooth)
value_loss   = keras.losses.MeanSquaredError()
metrics_dict = {'policy': ['categorical_accuracy'], 'value': ['mae']}

model.compile(optimizer=optimizer,
              loss={'policy': policy_loss, 'value': value_loss},
              loss_weights={'policy': 1.0, 'value': 1.0},
              metrics=metrics_dict)

ckpt_path = os.path.join(output_dir, f"{model_name}_best.keras")
csv_path  = os.path.join(output_dir, f"{model_name}_training_log.csv")

callbacks_common = [
    keras.callbacks.ModelCheckpoint(filepath=ckpt_path, monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
    keras.callbacks.CSVLogger(csv_path, append=True),
]

# ================================================================
# 🏋️‍♂️ ENTRAÎNEMENT (val tous les 5 epochs)
# ================================================================
history_all = {
    "loss": [], "policy_categorical_accuracy": [], "value_mae": [],
    "val_epochs": [], "val_loss": [], "val_policy_categorical_accuracy": [], "val_value_mae": []
}

for i in range(1, epochs + 1):
    print(f"\n--- Époque {i}/{epochs} ---")
    golois.getBatch(input_data, policy, value, end, groups, i * N)

    do_val = (i % 5 == 0) or (i == epochs)
    val_data, callbacks = None, []
    if do_val:
        golois.getValidation(input_data, policy, value, end)
        val_data = (input_data, [policy, value])
        callbacks = callbacks_common

    hist = model.fit(
        input_data, [policy, value],
        epochs=1, batch_size=batch, verbose=1,
        validation_data=val_data,
        callbacks=callbacks
    )

    history_all["loss"].append(hist.history.get("loss", [None])[0])
    history_all["policy_categorical_accuracy"].append(hist.history.get("policy_categorical_accuracy", [None])[0])
    history_all["value_mae"].append(hist.history.get("value_mae", [None])[0])

    if do_val:
        history_all["val_epochs"].append(i)
        history_all["val_loss"].append(hist.history.get("val_loss", [None])[0])
        history_all["val_policy_categorical_accuracy"].append(hist.history.get("val_policy_categorical_accuracy", [None])[0])
        history_all["val_value_mae"].append(hist.history.get("val_value_mae", [None])[0])

    if i % 5 == 0:
        gc.collect()
        print("🧹 GC.")

final_path = os.path.join(output_dir, f"{model_name}_final.keras")
model.save(final_path)
print(f"\n💾 Sauvegardes :\n - Best: {ckpt_path}\n - Final: {final_path}")
print("✅ Entraînement terminé.")


✅ GPU activé (memory growth).
🧾 Modèle : 20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001
📂 Dossier : output/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001
🔍 Test golois.getValidation()…
✅ golois prêt.



Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ board (InputLayer)  │ (None, 19, 19,    │          0 │ -                 │
│                     │ 31)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_40 (Conv2D)  │ (None, 19, 19,    │      3,410 │ board[0][0]       │
│                     │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        440 │ conv2d_40[0][0]   │
│ (BatchNormalizatio… │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_8 (ReLU)      │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_52 │ (None, 19, 19,    │     13,090 │ re_lu_8[0][0]     │
│ (SeparableConv2D)   │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        440 │ separable_conv2d… │
│ (BatchNormalizatio… │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_9 (ReLU)      │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_53 │ (None, 19, 19,    │     13,090 │ re_lu_9[0][0]     │
│ (SeparableConv2D)   │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 19, 19,    │        440 │ separable_conv2d… │
│ (BatchNormalizatio… │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_10 (ReLU)     │ (None, 19, 19,    │          0 │ batch_normalizat… │
│                     │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 110)       │          0 │ re_lu_10[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_81 (Dense)    │ (None, 11)        │      1,221 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_82 (Dense)    │ (None, 110)       │      1,320 │ dense_81[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_46          │ (None, 1, 1, 110) │          0 │ dense_82[0][0]    │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_26         │ (None, 19, 19,    │          0 │ re_lu_10[0][0],   │
│ (Multiply)          │ 110)              │            │ reshape_46[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_41 (Add)        │ (None, 19, 19,    │          0 │ re_lu_8[0][0],    │
│                     │ 110)              │            │ multiply_26[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_11 (ReLU)     │ (None, 19, 19,    │          0 │ add_41[0][0]      │
│                     │ 110)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 99,932 (390.36 KB)

 Trainable params: 98,392 (384.34 KB)

 Non-trainable params: 1,540 (6.02 KB)

🔢 Total params (Keras): 99932

--- Époque 1/60 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 51s 277ms/step - loss: 6.8163 - policy_categorical_accuracy: 0.0066 - policy_loss: 5.8116 - value_loss: 0.9240 - value_mae: 0.8007

--- Époque 2/60 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 5.5876 - policy_categorical_accuracy: 0.0247 - policy_loss: 5.3321 - value_loss: 0.1751 - value_mae: 0.3407

--- Époque 3/60 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 5.3237 - policy_categorical_accuracy: 0.0444 - policy_loss: 5.0902 - value_loss: 0.1534 - value_mae: 0.3229

--- Époque 4/60 ---
79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 5.0296 - policy_categorical_accuracy: 0.0855 - policy_loss: 4.8063 - value_loss: 0.1433 - value_mae: 0.3158

--- Époque 5/60 ---
78/79 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 4.8702 - policy_categorical_accuracy: 0.1076 - policy_loss: 4.6578 - value_loss: 0.1326 - value_mae: 0.3027
Epoch 1: val_loss improved from inf to 4.60657, saving model to output/20251110_Ada

# Sauvegarde

In [19]:
# ================================================================
#  SAUVEGARDE + VISUALISATION +  LISSAGE + ZIP DOWNLOAD
# ================================================================

import json
import matplotlib.pyplot as plt
from google.colab import files
import numpy as np
import os
import shutil

# 🔹 Création du dossier (si manquant)
os.makedirs(output_dir, exist_ok=True)

# 🔹 Sauvegarde du modèle (.h5 & .keras)
h5_path = os.path.join(output_dir, f"{model_name}.h5")
keras_path = os.path.join(output_dir, f"{model_name}.keras")
hist_json = os.path.join(output_dir, "history.json")

model.save(h5_path)
model.save(keras_path)
print(f"✅ Modèles sauvegardés :\n - {h5_path}\n - {keras_path}")

# 🔹 Sauvegarde de l’historique complet
with open(hist_json, "w") as f:
    json.dump(history_all, f)
print(f"📘 Historique sauvegardé : {hist_json}")

# ================================================================
# 📊 VISUALISATION DES COURBES
# ================================================================
plt.style.use("seaborn-v0_8-whitegrid")

def save_plot(fig, name):
    fig.savefig(os.path.join(output_dir, name), dpi=300, bbox_inches='tight')

epochs_range = range(1, len(history_all.get("loss", [])) + 1)

# --- 1️⃣ Courbe de perte (Loss)
if history_all.get("loss"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["loss"], label='Training Loss', linewidth=2)
    if history_all.get("val_loss"):
        ax.plot(history_all["val_epochs"], history_all["val_loss"], label='Validation Loss', linewidth=2)
    ax.set_title(f"{model_name} - Loss")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Loss")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "loss_curve.png")
    plt.close(fig)

# --- 2️⃣ Policy Accuracy
if history_all.get("policy_categorical_accuracy"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["policy_categorical_accuracy"], label='Policy Accuracy', linewidth=2)
    if history_all.get("val_policy_categorical_accuracy"):
        ax.plot(history_all["val_epochs"], history_all["val_policy_categorical_accuracy"], '--', label='Val Policy Acc', linewidth=2)
    ax.set_title(f"{model_name} - Policy Accuracy")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "policy_accuracy.png")
    plt.close(fig)

# --- 3️⃣ Top-5 Accuracy
if history_all.get("policy_top5"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["policy_top5"], label='Top-5 Accuracy', linewidth=2)
    if history_all.get("val_policy_top5"):  # ✅ clé optionnelle
        ax.plot(history_all["val_epochs"], history_all["val_policy_top5"], '--', label='Val Top-5', linewidth=2)
    ax.set_title(f"{model_name} - Top-5 Accuracy")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "top5_accuracy.png")
    plt.close(fig)

# --- 4️⃣ Value MAE
if history_all.get("value_mae"):
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(epochs_range, history_all["value_mae"], label='Value MAE', linewidth=2)
    if history_all.get("val_value_mae"):
        ax.plot(history_all["val_epochs"], history_all["val_value_mae"], '--', label='Val Value MAE', linewidth=2)
    ax.set_title(f"{model_name} - Value MAE")
    ax.set_xlabel("Epochs"); ax.set_ylabel("MAE")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.6)
    save_plot(fig, "value_mae.png")
    plt.close(fig)

# ================================================================
# 📈 LISSAGE (Policy Accuracy)
# ================================================================
def moving_average(data, window=5):
    data = np.array(data)
    if len(data) < window:
        return data
    return np.convolve(data, np.ones(window)/window, mode='valid')

if history_all.get("policy_categorical_accuracy"):
    smooth_acc = moving_average(history_all["policy_categorical_accuracy"], 5)
    epochs_smooth = np.arange(5, len(history_all["policy_categorical_accuracy"]) + 1)
    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(range(1, len(history_all["policy_categorical_accuracy"]) + 1),
            history_all["policy_categorical_accuracy"], alpha=0.4, label='Brut', linewidth=1.5)
    ax.plot(epochs_smooth, smooth_acc, color='red', linewidth=2.5,
            label='Moyenne glissante (5)')
    ax.set_title(f"{model_name} - Policy Accuracy (Lissée)")
    ax.set_xlabel("Epochs"); ax.set_ylabel("Accuracy")
    ax.legend(); ax.grid(True, linestyle='--', alpha=0.8)
    save_plot(fig, "policy_acc_smooth.png")
    plt.close(fig)

# ================================================================
# 🧾 EXPORT DU RÉSUMÉ DES MÉTRIQUES
# ================================================================
summary_path = os.path.join(output_dir, "metrics_summary.txt")
with open(summary_path, "w") as f:
    f.write(f"Model: {model_name}\n")
    f.write(f"Epochs: {len(history_all.get('loss', []))}\n\n")
    if history_all.get("loss"):
        f.write(f"Final Training Loss: {history_all['loss'][-1]:.4f}\n")
    if history_all.get("val_loss"):
        f.write(f"Final Validation Loss: {history_all['val_loss'][-1]:.4f}\n")
    if history_all.get("policy_categorical_accuracy"):
        f.write(f"Final Policy Accuracy: {history_all['policy_categorical_accuracy'][-1]:.4f}\n")
    if history_all.get("val_policy_categorical_accuracy"):
        f.write(f"Final Val Policy Accuracy: {history_all['val_policy_categorical_accuracy'][-1]:.4f}\n")
    if history_all.get("value_mae"):
        f.write(f"Final Value MAE: {history_all['value_mae'][-1]:.4f}\n")
    if history_all.get("val_value_mae"):
        f.write(f"Final Val Value MAE: {history_all['val_value_mae'][-1]:.4f}\n")
    if history_all.get("policy_top5"):
        f.write(f"Final Policy Top-5: {history_all['policy_top5'][-1]:.4f}\n")

print(f"\n✅ Fichiers et graphiques enregistrés dans : {output_dir}")

# ================================================================
# 📦 ZIP + TÉLÉCHARGEMENT AUTOMATIQUE
# ================================================================
zip_path = f"{output_dir}.zip"
shutil.make_archive(output_dir, 'zip', output_dir)
print(f"📦 Dossier compressé : {zip_path}")

try:
    files.download(zip_path)
    print(f"📥 Téléchargement du dossier complet lancé avec succès ✅")
except Exception as e:
    print(f"⚠️ Téléchargement automatique ignoré ({e})")


✅ Modèles sauvegardés :
 - output/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001.h5
 - output/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001.keras
📘 Historique sauvegardé : output/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001/history.json

✅ Fichiers et graphiques enregistrés dans : output/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001
📦 Dossier compressé : output/20251110_AdamH_GoSepResNet_100k_ep60_bs128_lr0.0001.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Téléchargement du dossier complet lancé avec succès ✅
